In [1]:
#-*- coding: EUC-KR -*-

%matplotlib inline 

import numpy as np 
import pandas as pd
import skimage.io as io 
import matplotlib.pyplot as plt 
import pylab 
import json
from tqdm import tqdm
import glob
import os
import seaborn as sns
from pathlib import Path
import base64
import cv2
import scipy 
from joblib import Parallel , delayed

pylab.rcParams['figure.figsize'] = (8.0, 10.0)

rootpath = "D:/"

In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [4]:
device="cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


# Yolov5 train.py

In [6]:
import os
os.chdir("D:/yolov5")

In [7]:
path= "D:/"

In [8]:
path#D:\yolov5\data

'D:/'

In [9]:
from glob import glob

train_list = glob(path+'yolov5/data/images/train/*.jpg')
test_list =  glob(path+'yolov5/data/images/valid/*.jpg')

print(len(train_list), len(test_list))

5729 1844


In [10]:
with open(path+'/yolov5/train.txt', 'w') as f:
    f.write('\n'.join(train_list) + '\n')

with open(path+'/yolov5/valid.txt', 'w') as f:
    f.write('\n'.join(test_list) + '\n')

In [11]:
import yaml

In [20]:
from glob import glob

# train_list = glob(path+'yolov5/data/images/train/*.jpg')
# test_list =  glob(path+'yolov5/data/images/valid/*.jpg')

train_list = glob(path+'road/images/train/*.jpg')
test_list =  glob(path+'road/images/valid/*.jpg')



print(len(train_list), len(test_list))

5729 1844


In [21]:
# with open(path+'/yolov5/train.txt', 'w') as f:
#     f.write('\n'.join(train_list) + '\n')

# with open(path+'/yolov5/valid.txt', 'w') as f:
#     f.write('\n'.join(test_list) + '\n')

with open(path+'/yolov5/train_road.txt', 'w') as f:
    f.write('\n'.join(train_list) + '\n')

with open(path+'/yolov5/valid_road.txt', 'w') as f:
    f.write('\n'.join(test_list) + '\n')

In [22]:
import yaml

In [24]:
with open('D:/yolov5/data/hyps/hyp.scratch.yaml', 'r',encoding="utf-8") as f:
    data = yaml.safe_load(f)

In [25]:
data

{'lr0': 0.01,
 'lrf': 0.1,
 'momentum': 0.937,
 'weight_decay': 0.0005,
 'warmup_epochs': 3.0,
 'warmup_momentum': 0.8,
 'warmup_bias_lr': 0.1,
 'box': 0.05,
 'cls': 0.5,
 'cls_pw': 1.0,
 'obj': 1.0,
 'obj_pw': 1.0,
 'iou_t': 0.2,
 'anchor_t': 4.0,
 'fl_gamma': 0.0,
 'hsv_h': 0.015,
 'hsv_s': 0.7,
 'hsv_v': 0.4,
 'degrees': 0.0,
 'translate': 0.1,
 'scale': 0.5,
 'shear': 0.0,
 'perspective': 0.0,
 'flipud': 0.0,
 'fliplr': 0.5,
 'mosaic': 1.0,
 'mixup': 0.0,
 'copy_paste': 0.0}

In [26]:
data['train'] = path+'yolov5/train_road.txt'
data['val'] = path+'yolov5/valid_road.txt'

    
data['box'] = 0.05  # box loss gain 

data['cls_pw']= 1.0  # cls BCELoss positive_weight 
data['obj']= 1.0  # obj loss gain (scale with pixels) 
data['obj_pw'] = 1.0  # obj BCELoss positive_weight 
data['iou_t'] = 0.30  # IoU training threshold 
    
data['hsv_h']= 0.8  # image HSV-Hue augmentation (fraction) 
data['hsv_s']= 0.8  # image HSV-Saturation augmentation (fraction) 
data['hsv_v']= 0.8  # image HSV-Value augmentation (fraction) 
data['degrees']= 0.1  # image rotation (+/- deg) 
data['translate'] = 0.1  # image translation (+/- fraction) 
data['scale'] = 0.6  # image scale (+/- gain) 
data['shear'] = 0.01 # image shear (+/- deg) 


data['nc']=8

data['names']=['sidewalk','sidewalk_byc','crosswalk','byc_road','car_road','centerline','safeline','stopline']

In [27]:


with open(path+'yolov5/data_road.yaml', 'w') as f:
    yaml.dump(data, f)

print(data)

{'lr0': 0.01, 'lrf': 0.1, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'warmup_bias_lr': 0.1, 'box': 0.05, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.3, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.8, 'hsv_s': 0.8, 'hsv_v': 0.8, 'degrees': 0.1, 'translate': 0.1, 'scale': 0.6, 'shear': 0.01, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0, 'train': 'D:/yolov5/train_road.txt', 'val': 'D:/yolov5/valid_road.txt', 'nc': 8, 'names': ['sidewalk', 'sidewalk_byc', 'crosswalk', 'byc_road', 'car_road', 'centerline', 'safeline', 'stopline']}


In [15]:
os.chdir("D:/yolov5")
!python train.py --img 640 --batch 8 --epochs 8 --data ./data_road.yaml --cfg ./models/yolov5s.yaml --iou-thres 0.7 --augment --weights yolov5s.pt --name road_yolov5s_results --device 0     

github:  YOLOv5 is out of date by 2 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.

train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=./data.yaml, hyp=data\hyps\hyp.scratch.yaml, epochs=1, batch_size=16, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs\train, name=2viol_yolov5s_results, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLOv5  v6.0-111-g94d8fec torch 1.10.0+cpu CPU

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic


Weights & Biases: run 'pip install wandb' to automatically track and visualize YOLOv5  runs (RECOMMENDED)
module 'signal' has no attribute 'SIGALRM'
Optimizer stripped from runs\train\2viol_yolov5s_results\weights\last.pt, 14.5MB
Optimizer stripped from runs\train\2viol_yolov5s_results\weights\best.pt, 14.5MB



                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  8                -1  1   1182720  

In [17]:
val_img_path="C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/RAW데이터/대용량데이터/PM 사고위험 영상데이터/6 신호 위반/자전거/위반/T000122_000_1_1_6.mp4"

In [18]:
!python detect.py --weights D:/yolov5/runs/train/2viol_yolov5s_results/weights/best.pt --img 416 --conf 0.3 --source "{val_img_path}" --save-txt


detect: weights=['D:/yolov5/runs/train/2viol_yolov5s_results/weights/best.pt'], source=C:/Users/user/Desktop/PM이륜차 프로젝트/데이터/RAW데이터/대용량데이터/PM 사고위험 영상데이터/6 신호 위반/자전거/위반/T000122_000_1_1_6.mp4, imgsz=[416, 416], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  v6.0-111-g94d8fec torch 1.10.0+cpu CPU

Fusing layers... 
Model Summary: 213 layers, 7120702 parameters, 0 gradients, 16.1 GFLOPs
video 1/1 (1/453) C:\Users\user\Desktop\PM이륜차 프로젝트\데이터\RAW데이터\대용량데이터\PM 사고위험 영상데이터\6 신호 위반\자전거\위반\T000122_000_1_1_6.mp4: 256x416 2 sidewalks, 2 car_roads, Done. (0.067s)
video 1/1 (2/453) C:\Users\user\Desktop\PM이륜차 프로젝트\데이터\RAW데이터\대용량데이터\PM 사고위험 영상데이터\6 신호 위반\자전거\위반\T000122_000_1_1_6.mp4: 256x416 2 sidewalks, 2 car

In [73]:
!python detect.py --weights D:/yolov5/runs/train/viol_yolov5s_results6/weights/best.pt --img 640 --conf 0.3 --source "D:/PM사고위험영상데이터/6 신호 위반/자전거/정상/T000219_000_0_1_6.mp4" --save-txt --save-crop


^C
